In [1]:
pip install transformers

In [2]:
pip install torch

In [3]:
pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 95.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [5]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.manifold import MDS
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [7]:
# Define the verb of interest
agree_on = 'agree'
agree_with = 'agree'
agree_that = 'agree'
agree_to = 'agree'

In [8]:
import pandas as pd
import numpy as np
df = pd.read_excel('Data_for_MDS_with_indices.xlsx')
df

,Number of hit,Text ID,Concordance line,Tagged concordance line,URL,Match-begin corpus position,Match-end corpus position,pos_tag,verb,row_in_verb
0,1,A79,"contrast to most plants , they breathe in oxyg...","contrast_NN1 to_PRP most_DT0 plants_NN2 ,_PUN ...",https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,3985690,3985692,give_out_NN1,give_out,0
1,2,AP8,Kendal of &lsquo; hosiers attending Markets to...,Kendal_NP0 of_PRF &lsquo;_PUQ hosiers_NN2 atte...,https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,12842533,12842535,give_out_NN1,give_out,1
2,3,C8P,", such as Merlin and Reg Webb Entertainments ,...",",_PUN such_DT0 as_PRP Merlin_NP0 and_CJC Reg_N...",https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,23841186,23841188,give_out_NN1,give_out,2
3,4,CH6,&lsquo; If we deal with it ourselves we could ...,&lsquo;_PUQ If_CJS we_PNP deal_VVB with_PRP it...,https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,34531644,34531646,give_out_NN1,give_out,3
4,5,ED3,"get better . Equally , although a volunteer ca...","get_VVI better_AJC ._PUN Equally_AV0 ,_PUN alt...",https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,45732630,45732632,give_out_NN1,give_out,4
...,...,...,...,...,...,...,...,...,...,...
1087,96,CCU,of the Work . The Publisher and the Proprietor...,of_PRF the_AT0 Work_NN1 ._PUN The_AT0 Publishe...,https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,29402333,29402336,agree_on_AT0,agree_on,95
1088,97,FCG,proceedings before the family proceedings cour...,proceedings_NN2 before_PRP-CJS the_AT0 family_...,https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,54018008,54018011,agree_on_AT0,agree_on,96
1089,98,AA4,"party , said all political forces and ethnic g...","party_NN1 ,_PUN said_VVD all_DT0 political_AJ0...",https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,5087966,5087969,agree_on_AT0,agree_on,97
1090,99,HKS,it would not participate because of India 's r...,it_PNP would_VM0 not_XX0 participate_VVI becau...,https://cqpweb.lancs.ac.uk/bncxmlweb/context.p...,81118896,81118899,agree_on_AT0,agree_on,98


In [9]:
agree_on_sentences = df[df['verb'] == 'agree_on']['Concordance line'].tolist()
agree_with_sentences = df[df['verb'] == 'agree_with']['Concordance line'].tolist()
agree_that_sentences = df[df['verb'] == 'agree_that']['Concordance line'].tolist()
agree_to_sentences = df[df['verb'] == 'agree_to']['Concordance line'].tolist()

In [ ]:
agree_on_sentences

In [24]:
import re
import string

def clean_sentences(sentences):
    # Define a translation table that maps punctuation to None
    translator = str.maketrans('', '', string.punctuation)

    cleaned_sentences = []
    for sentence in sentences:
        cleaned_sentence = sentence.strip()  # Remove leading and trailing whitespace
        cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence)  # Replace multiple spaces with a single space
        cleaned_sentence = cleaned_sentence.lower()  # Convert to lowercase
        cleaned_sentence = cleaned_sentence.translate(translator)  # Remove punctuation
        cleaned_sentences.append(cleaned_sentence)

    return cleaned_sentences

In [19]:
#clean
cleaned_agree_on_sentences = clean_sentences(agree_on_sentences)
cleaned_agree_with_sentences = clean_sentences(agree_with_sentences)
cleaned_agree_that_sentences = clean_sentences(agree_that_sentences)
cleaned_agree_to_sentences = clean_sentences(agree_to_sentences)

In [ ]:
cleaned_agree_to_sentences

In [21]:
# Function to process a sentence and get hidden state for a given verb
def process_sentence(sentence, verb):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    hidden_state_verb = outputs.hidden_states[0][0, inputs['input_ids'][0].tolist().index(tokenizer.vocab[verb]), :]
    return hidden_state_verb

In [26]:
# Collect hidden states
agree_on_hidden_states = [process_sentence(sentence, agree_on).numpy() for sentence in cleaned_agree_on_sentences]
agree_with_hidden_states = [process_sentence(sentence, agree_with).numpy() for sentence in cleaned_agree_with_sentences]
agree_that_hidden_states = [process_sentence(sentence, agree_that).numpy() for sentence in cleaned_agree_that_sentences]
agree_to_hidden_states = [process_sentence(sentence, agree_to).numpy() for sentence in cleaned_agree_to_sentences]

In [33]:
len(agree_to_hidden_states[0])

768